In [1]:
import numpy as np
import tensorflow as tf

In [2]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, :-1]
y_data = xy[:, [-1]] # xy[:, -1]과는 다름에 유의!
print(x_data, y_data)
print(x_data.shape, y_data.shape)

[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]] [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]]
(25, 3) (25, 1)


In [3]:
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None ,1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], {X: x_data, Y:y_data})
#     if step%10 == 0:
#         print(step, "cost: ", cost_val, ", hy: ", hy_val)

print(sess.run(hypothesis, {X: [[100, 70, 101]]}))

[[193.24379]]


In [4]:
filename_queue = tf.train.string_input_producer(['data-01-test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TextLineDataset`.


In [18]:
train_x_batch, train_y_batch = tf.train.batch(
    [xy[:-1], xy[-1:]], # y_data 에서 xy[-1] 이 아니라 xy[-1:]임에 유의!
    batch_size=10)

In [21]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
    [cost, hypothesis, train],
    {X:x_batch, Y:y_batch})
    if step % 10 == 0:
        print(step, "cost: ", cost_val)
    
coord.request_stop()
coord.join(threads)

0 cost:  575.3823
10 cost:  73.72204
20 cost:  72.040436
30 cost:  71.72969
40 cost:  71.429695
50 cost:  71.131226
60 cost:  70.834236
70 cost:  70.53872
80 cost:  70.244576
90 cost:  69.95195
100 cost:  69.66078
110 cost:  69.3709
120 cost:  69.0825
130 cost:  68.79555
140 cost:  68.51003
150 cost:  68.225845
160 cost:  67.94305
170 cost:  67.66158
180 cost:  67.38172
190 cost:  67.10309
200 cost:  66.825874
210 cost:  66.549965
220 cost:  66.27536
230 cost:  66.00206
240 cost:  65.73035
250 cost:  65.45965
260 cost:  65.19044
270 cost:  64.922485
280 cost:  64.6559
290 cost:  64.3907
300 cost:  64.126625
310 cost:  63.863934
320 cost:  63.602577
330 cost:  63.342358
340 cost:  63.08355
350 cost:  62.82595
360 cost:  62.56955
370 cost:  62.314568
380 cost:  62.06069
390 cost:  61.808067
400 cost:  61.556602
410 cost:  61.30655
420 cost:  61.057667
430 cost:  60.809834
440 cost:  60.563305
450 cost:  60.318092
460 cost:  60.073944
470 cost:  59.831036
480 cost:  59.589283
490 cost:  5